In [40]:
import re
import numpy as np
import pandas as pd
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [67]:
df = pd.read_csv('Dataset/DataScientist.csv')
df2 = pd.read_csv('Dataset/DataAnalyst.csv')
df = df.drop(columns=['index'])
df = df.drop(columns=['Unnamed: 0'])
df2 = df2.drop(columns=['Unnamed: 0'])

df_final = pd.concat([df, df2], ignore_index=True).drop_duplicates()
df_final = df_final.drop(columns=['Easy Apply'])
print(df_final.shape)
df_final.head()

(6161, 14)


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1
3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades"
4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5"


In [68]:
print(df_final['Job Description'].apply(type).value_counts())

Job Description
<class 'str'>    6161
Name: count, dtype: int64


In [74]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


def nettoyer(text):
  if isinstance(text, str):
    text = text.lower()
    # Supprimer les caractères spéciaux
    text = re.sub(r'(http\S+|[^a-zA-Z\s])', '', text)
    # Tokenisation
    words = nltk.word_tokenize(text)
    # Suppression des mots vides et lemmatisation
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    text = ' '.join(words)
    # Reconstituer le texte prétraité
    return text
  else:
    return np.nan

df_final['Job Description'] = df_final['Job Description'].apply(nettoyer)
df_final['Company Name'] = df_final['Company Name'].apply(nettoyer)
df_final['Revenue'].replace(-1, np.nan)
df_final['Founded'].replace(-1, np.nan)
df_final.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),hopper hopper mission make booking travel fast...,3.5,hopper,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),noom use scientifically proven method help use...,4.5,noom u,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1
2,Data Science Manager,$111K-$181K (Glassdoor est.),decodem data science manager job description h...,-1.0,decodem,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1
3,Data Analyst,$111K-$181K (Glassdoor est.),sapphire digital seek dynamic driven midlevel ...,3.4,sapphire digital,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades"
4,"Director, Data Science",$111K-$181K (Glassdoor est.),director data science description edelman inte...,3.4,united entertainment group,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5"
